In [1]:
#inicamos spark
import findspark
findspark.init()

import pandas as pd
import pyspark

from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import *


spark = SparkSession.builder.getOrCreate()


In [2]:
df = spark.read.csv("Datasets/world-happiness-report-2021.csv", sep=",", header=True, inferSchema=True)

1. ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” 
mayor número más feliz es el país) 

In [3]:
# Seleccionamos el primero
df_pri = df.select("Country name", "Ladder score").orderBy(desc("Ladder score")).first()[0]
print(f"El pais con mayor indice de felicidad en 2021 es {df_pri}.")

El pais con mayor indice de felicidad en 2021 es Finland.


2. ¿Cuál es el país más “feliz” del 2021 por continente según la data? 

In [4]:

df.orderBy(desc("Ladder score")).groupBy("Regional indicator").agg(first("Country name").alias("Country name"),first("Ladder score").alias("Max Ladder Score")).show()

+--------------------+--------------------+----------------+
|  Regional indicator|        Country name|Max Ladder Score|
+--------------------+--------------------+----------------+
|Central and Easte...|      Czech Republic|           6.965|
|Commonwealth of I...|          Uzbekistan|           6.179|
|           East Asia|Taiwan Province o...|           6.584|
|Latin America and...|          Costa Rica|           7.069|
|Middle East and N...|              Israel|           7.157|
|North America and...|         New Zealand|           7.277|
|          South Asia|               Nepal|           5.269|
|      Southeast Asia|           Singapore|           6.377|
|  Sub-Saharan Africa|           Mauritius|           6.049|
|      Western Europe|             Finland|           7.842|
+--------------------+--------------------+----------------+



3. ¿Cuál es el país que más veces ocupó el primer lugar en todos los años? 

In [5]:
#Leemeos el segundo dataset
df_2 = spark.read.csv("Datasets/world-happiness-report.csv", sep=",", header=True, inferSchema=True)

In [6]:
#nos quedamos con las columnas que nos interesan
df_2_filter = df_2.select("Country name","year","Life Ladder","Log GDP per capita","Healthy life expectancy at birth")


In [7]:
#anyadimos columna year al primer dataset
df_2021 = df.withColumn("year", lit(2021))

In [8]:
#nos quedamos con las columnas que nos interesan
df_2021 = df_2021.select("Country name","year","Ladder score","Logged GDP per capita","Healthy life expectancy")

In [9]:
#Cambiamos el combre de la columnas para que coincida con el primer dataset
df_2021 = df_2021.withColumnRenamed("Ladder score","Life Ladder").withColumnRenamed("Logged GDP per capita","Log GDP per capita").withColumnRenamed("Healthy life expectancy","Healthy life expectancy at birth")

In [10]:
#Unimos los dos dataframe con las columnas que nos interesan
df_union = df_2_filter.union(df_2021).distinct()

In [14]:

#df_union.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("union.csv")

In [11]:
#rankeamos para ello havemos una particion por anyo
df_ranked = df_union.withColumn("rank", rank().over(Window.partitionBy("year").orderBy(desc("Life Ladder"), "year")))

In [12]:

#vemos los 5 primeros paises
df_ranked.filter(df_ranked["rank"] == 1).groupBy("Country Name").count().orderBy(desc("count")).show(5)

+------------+-----+
|Country Name|count|
+------------+-----+
|     Finland|    7|
|     Denmark|    7|
|      Norway|    1|
| Switzerland|    1|
|      Canada|    1|
+------------+-----+



Como se observa hay empate en el primer puesto entre Finladia y Dinamarca

4. ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020? 

In [13]:
#filtramos primero las columnas que nos interesan
df_2_filter2 = df_2.filter(df_2["year"] == 2020).select("Country name","year","Life Ladder","Log GDP per capita")

In [14]:
#vemos que pais tiene el mayor  GDP en 2020
gdp_2020 = df_2_filter2.orderBy(desc("Log GDP per capita")).first()[0]

#print(f"El pais con mayor GDP en 2020 es {gdp_2020}.")

In [15]:
#rankeamos por life ladder
filter2_ranked = df_2_filter2.withColumn("rank", rank().over(Window.orderBy(desc("Life Ladder"))))


In [16]:
#vemos el puesto de irlanda
lf_gdp_2020 = filter2_ranked.select("rank").where(filter2_ranked["Country name"] == gdp_2020).first()[0]
print(f"El pais con mayor GDP en 2020 fue {gdp_2020}, el cual tenia el puesto de felicidad numero {lf_gdp_2020}.")

El pais con mayor GDP en 2020 fue Ireland, el cual tenia el puesto de felicidad numero 13.


5. ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó 
o disminuyó? 

In [17]:
#Agrupamos por anyo y calculamos la media

avg_GDP = df_union.groupBy("year").agg(avg("Log GDP per capita").alias("avg Log GDP per capita"))

In [18]:
#calculamos el porcentaje de variacion

avg_GDP_2020= avg_GDP.select("avg Log GDP per capita").where(avg_GDP["year"] == 2020).collect()[0][0]

avg_GDP_2021= avg_GDP.select("avg Log GDP per capita").where(avg_GDP["year"] == 2021).collect()[0][0]

pct_cambio = ((avg_GDP_2021 - avg_GDP_2020) / avg_GDP_2020) * 100
print(f"La variacion en porcentaje de la media del GDP del 2020 al 2021 es {pct_cambio:.2f}% por lo que en el 2021 disminuyo.")

La variacion en porcentaje de la media del GDP del 2020 al 2021 es -3.27% por lo que en el 2021 disminuyo.


6. ¿Cuál es el país con mayor expectativa de vida (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia ese indicador en el 2019? 

In [19]:
#vemos que pais tiene el mayor Healthy life expectancy at birth
Life_expectancy_2021 = df_union.select("Country name", "year").filter(df_union["year"] == 2021).orderBy(desc("Healthy life expectancy at birth")).first()[0]
print(f"El pais con mayor esperanza de vida en el 2021 es {Life_expectancy_2021}.")

El pais con mayor esperanza de vida en el 2021 es Singapore.


In [20]:
#vemos que esperanza de vida tenia en el 2019
Life_expectancy_2019 = df_union.select("Country name", "year","Healthy life expectancy at birth").filter((df_union["year"] == 2019) & (df_union["Country name"] == Life_expectancy_2021)).first()[2]
print(f"{Life_expectancy_2021} tenia una esperanza de vida en el 2019 de {Life_expectancy_2019} años.")

Singapore tenia una esperanza de vida en el 2019 de 77.1 años.
